In [20]:
import nets 
import scipy.io as scio
import numpy as np
import os
from tqdm.auto import tqdm, trange
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F


In [21]:
model =  torch.load('/Users/sheeta/Dropbox/2021_GAN_ambient light removal/CNN_old code/codes/experiments/Task1_CNN/model_latest.pth', map_location='cpu')
model.eval()

/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/anaconda3/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attri

CNN_CE(
  (feat_extractor): Sequential(
    (0): Conv1d(1, 16, kernel_size=(11,), stride=(5,), bias=False)
    (1): ReLU()
    (2): Conv1d(16, 32, kernel_size=(7,), stride=(3,), bias=False)
    (3): ReLU()
  )
  (feat_mapping): Sequential(
    (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): ReLU()
    (2): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (3): ReLU()
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool1d(output_size=1)
    (1): Flatten()
    (2): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [4]:
os.chdir('/Users/sheeta/Desktop/learn_torch/Task1')
lightdata = scio.loadmat('Room_light.mat')
lightdata = torch.Tensor(lightdata['light'])
light = torch.transpose(lightdata,0,1)[0]

FileNotFoundError: [Errno 2] No such file or directory: 'Room_light.mat'

In [5]:
#Get data of light and npc spectrum
os.chdir('/Users/sheeta/Desktop/learn_torch')

lightdata = scio.loadmat('Room_light.mat')
lightdata = torch.Tensor(lightdata['light'])
light = torch.transpose(lightdata,0,1)
database = scio.loadmat('NPC_database.mat')
npcdata = torch.Tensor(database['NPC_spec'])
controldata = torch.Tensor(database['control_spec'])

npc = torch.transpose(npcdata,0,1)
noise_zero_npc = torch.where(light[1]>0.13, torch.zeros_like(npc), npc)
control = torch.transpose(controldata,0,1)



In [6]:
#先zero，后修复的光谱
output_total = np.load("output_total.npy",allow_pickle = True)
output_control = output_total[:1760]
output_npc = output_total[1761:]

In [54]:
pred_test = np.load("output_test.npy", allow_pickle = True)
ori_test = np.load("golden_test.npy",allow_pickle = True)

In [50]:
len(ori_test)

1182

In [26]:
def acc(database):
    score = 0
    for i in range(len(database)):
        
        pred = model(torch.Tensor(database[i][0]).view(-1,1,779))
        pred = np.argmax(pred.detach().numpy())
        if pred == database[i][1]:
            score+=1
    acc = score/len(database) 

    return acc

In [28]:
def negacc(database):
    score = 0
    size = 0
    for i in range(len(database)):
        
        if database[i][1] ==0:
            size+=1
            pred = model(torch.Tensor(database[i][0]).view(-1,1,779))
            pred = np.argmax(pred.detach().numpy())

            if pred == database[i][1]:

                score+=1
    acc = score/size
    return acc

In [ ]:
def posiacc(database):
    score = 0
    size = 0
    for i in range(len(database)):
        
        if database[i][1] ==1:
            size+=1
            pred = model(torch.Tensor(database[i][0]).view(-1,1,779))
            pred = np.argmax(pred.detach().numpy())

            if pred == database[i][1]:

                score+=1
    acc = score/size
    return acc

In [73]:
#input:noise free spectrum
acc(ori_test)

0.8104906937394247

In [74]:
negacc(ori_test)

0.5125348189415042

In [75]:
negacc(pred_test)

0.49303621169916434

In [56]:
#inout: denoised spectrum
acc(pred_test)

0.8062605752961083

In [8]:
#get true positive of model use 'full of 0 data', acc = 0.18808777429467086
acc(noise_zero_npc)

0.18808777429467086

In [5]:
###use nasal model to predict mouth...
os.chdir('/Users/sheeta/Desktop/learn_torch/Task3')
#load raman
spec_mat= scio.loadmat('scale_spec')
spec = torch.Tensor(spec_mat['ans'])

#load label
label_mat= scio.loadmat('label')
label = torch.Tensor(label_mat['label'])


mouth = []
for i in range(len(label)):
    mouth.append(np.array([spec[i],label[i]],dtype=object))

In [27]:
#acc = 0.747268873308332
acc(mouth)

0.747268873308332

In [29]:

negacc(mouth)

0.9394965094140046

In [32]:
#
posiacc(mouth)

0.10099573257467995